In [2]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download()

from transformers import AutoTokenizer
tqdm.pandas()

In [13]:
path = '../Data/Raw/rcrwireless.csv'

df_origin = pd.read_csv(path)
df_origin['Date'] = pd.to_datetime(df_origin['Date'])
df_origin['Keywords'] = df_origin['Keywords'].apply(lambda x : eval(x))
df_origin['Category'] = df_origin['Category'].apply(lambda x : eval(x))
df_origin = df_origin[~df_origin['Text'].isnull()]

In [14]:
df_origin['Title'] = df_origin['Title'].apply(lambda x : x.lower())
df_origin['Text'] = df_origin['Text'].apply(lambda x : x.lower())

In [16]:
# model_checkpoint = "google/mt5-small"
# pipe = pipeline('summarization', model = model_checkpoint, max_length = 512, truncation = True)
# df_origin['SummarizedText'] = df_origin['Text'].progress_apply(lambda x : pipe(x))
df_origin['SummarizedText'] = df_origin['SummarizedText'].apply(lambda x : eval(x))
df_origin['SummarizedText'] = df_origin['SummarizedText'].apply(lambda x : x[0]['summary_text'])
df_origin['SummarizedText'] = df_origin['SummarizedText'].apply(lambda x : re.sub('<.*>', 'thing', x))

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
stop_words = set(stopwords.words('english'))

In [103]:
def stopWords(x):
    result = []
    x = word_tokenize(x)
    for word in x:
        if word not in stop_words:
            result.append(word)
    return ' '.join(result)

In [104]:
df_origin['Title__stop'] = df_origin['Title'].progress_apply(lambda x : stopWords(x))
df_origin['SummarizedText__stop'] = df_origin['SummarizedText'].progress_apply(lambda x : stopWords(x))

100%|██████████| 4470/4470 [00:00<00:00, 6540.48it/s]


In [105]:
df_origin.head(2)

,Title,Author,Date,Text,Keywords,Href,Category,SummarizedText,Title__stop,SummarizedText__stop
0,"next g alliance, india’s bharat 6g alliance in...",Juan Pedro Tomás,2023-09-11 11:21:18-05:00,atis’ next g alliance (nga) and india’s bhara...,"[6G, Featured, Networks]",https://www.rcrwireless.com/20230911/6g/nextg-...,[6G],thing to increase efficiency.,"next g alliance , india ’ bharat 6g alliance i...",thing increase efficiency .
1,south korea to start project to develop future...,Juan Pedro Tomás,2023-08-24 10:28:52-05:00,the government of south korea expects to start...,"[6G, Featured, Spectrum, Standards]",https://www.rcrwireless.com/20230824/6g/south-...,[6G],thing of 6G systems,south korea start project develop future 6g te...,thing 6G systems


In [106]:
lemmatizer = WordNetLemmatizer()
lancaster_stemmer = LancasterStemmer()

In [111]:
df_origin['PreprocessedTitle'] = df_origin['Title__stop'].progress_apply(lambda x : nltk.word_tokenize(x))
df_origin['PreprocessedTitle'] = df_origin['PreprocessedTitle'].progress_apply(lambda x : [lemmatizer.lemmatize(word) for word in x])
df_origin['PreprocessedTitle'] = df_origin['PreprocessedTitle'].progress_apply(lambda x : [lancaster_stemmer.stem(word) for word in x])

100%|██████████| 4470/4470 [00:01<00:00, 4184.57it/s]


In [112]:
# SummarizedText를 진행함에 있어, <extra_id_0> 이런 형식의 단어들을 thing으로 변화 필요
df_origin['PreprossedSummarizedText'] = df_origin['SummarizedText__stop'].progress_apply(lambda x : nltk.word_tokenize(x))
df_origin['PreprossedSummarizedText'] = df_origin['PreprossedSummarizedText'].progress_apply(lambda x : [lemmatizer.lemmatize(word) for word in x])
df_origin['PreprossedSummarizedText'] = df_origin['PreprossedSummarizedText'].progress_apply(lambda x : [lancaster_stemmer.stem(word) for word in x])

  0%|          | 0/4470 [00:00<?, ?it/s]

100%|██████████| 4470/4470 [00:00<00:00, 10309.43it/s]


In [113]:
df_origin['Preprocessed'] = df_origin.progress_apply(lambda x : x['PreprocessedTitle'] + x['PreprossedSummarizedText'] + x['Keywords'], axis = 1)

100%|██████████| 4470/4470 [00:00<00:00, 17804.12it/s]


In [1]:
df_origin.head(2)

NameError: name 'df_origin' is not defined

In [ ]:
df

In [115]:
df_origin.to_csv('../Data/Preprocessed/rcrwireless.csv', index = False)